# 02 - Treinamento e Avaliação de Modelos

**Objetivo:** Este notebook utiliza o conhecimento da análise exploratória para treinar, avaliar e comparar diferentes modelos de Machine Learning, com o objetivo de encontrar o classificador mais preciso e robusto para o nosso problema.

**Etapas:**
1.  **Configuração e Preparação dos Dados**: Carrega os dados e aplica a engenharia de features desenvolvida no notebook anterior.
2.  **Divisão de Dados**: Separa os dados em conjuntos de treino e teste para uma avaliação imparcial.
3.  **Seleção de Features**: Define um subconjunto de features para treinar um modelo mais simples e eficiente.
4.  **Divisão de Dados**: Separa os dados em conjuntos de treino e teste para uma avaliação imparcial.
5.  **Modelo de Base (Gaussian Naive Bayes)**: Treina e avalia o modelo atual para estabelecer uma linha de base de performance.
6.  **Experimentação com Modelos Avançados**: Treina e avalia `Random Forest` e `LightGBM` para buscar uma performance superior.
7.  **Comparação e Seleção**: Compara a performance de todos os modelos usando validação cruzada para escolher o melhor.
8.  **Análise de Importância das Features**: Investiga quais features mais contribuíram para a decisão do melhor modelo.
9.  **Treinamento e Exportação do Modelo Final**: Treina o modelo `GaussianNB` com todos os dados e o exporta no formato correto para uso no dashboard.

In [ ]:
import json
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import kurtosis, skew
from sqlalchemy import create_engine

# Modelos e Ferramentas de Avaliação
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

# --- CONFIGURAÇÕES ---

# Conexão com o banco de dados
DB_CONNECTION_STR = 'mysql+mysqlconnector://root:@localhost/iot_mpu6050'

# Parâmetros da Engenharia de Features
WINDOW_SIZE = 100  # Tamanho da janela deslizante
STEP = 20          # Passo da janela

# Diretório para salvar o modelo treinado
PATH_MODELS = 'output/models/'
os.makedirs(PATH_MODELS, exist_ok=True)

# Configurações visuais
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 5)

print("Ambiente configurado com sucesso!")

In [ ]:
# Esta função é a mesma do notebook 01_Analise_Exploratoria
# Ela transforma os dados brutos do sinal em features estatísticas

def extract_time_domain_features(series):
    """Calcula um conjunto de features de domínio do tempo para uma série de dados."""
    series = series.dropna()
    if series.empty:
        return pd.Series(dtype='float64')

    # Métricas básicas
    mean = series.mean()
    std = series.std()
    
    # Métricas de amplitude
    rms = np.sqrt(np.mean(series**2))
    peak = series.abs().max()
    root_amplitude = (np.mean(np.sqrt(series.abs())))**2
    mean_abs = series.abs().mean()

    # Fatores de forma (evitar divisão por zero)
    crest_factor = peak / rms if rms > 0 else 0
    shape_factor = rms / mean_abs if mean_abs > 0 else 0
    impulse_factor = peak / mean_abs if mean_abs > 0 else 0
    clearance_factor = peak / root_amplitude if root_amplitude > 0 else 0

    return pd.Series({
        'mean': mean,
        'std': std,
        'skew': skew(series),
        'kurtosis': kurtosis(series),
        'rms': rms,
        'peak': peak,
        'root_amplitude': root_amplitude,
        'crest_factor': crest_factor,
        'shape_factor': shape_factor,
        'impulse_factor': impulse_factor,
        'clearance_factor': clearance_factor
    })

print("Função extract_time_domain_features definida.")

In [ ]:
print("--- [1/9] Carregando e Processando Dados ---")

# 1. Carregar dados do banco
try:
    engine = create_engine(DB_CONNECTION_STR)
    query = "SELECT * FROM sensor_data WHERE fan_state IN ('LOW', 'MEDIUM', 'HIGH') ORDER BY timestamp ASC"
    df_raw = pd.read_sql(query, engine)
    print(f"✅ Dados brutos carregados: {len(df_raw)} linhas.")
except Exception as e:
    print(f"❌ ERRO AO CARREGAR DADOS: {e}")
    df_raw = pd.DataFrame()

# 2. Aplicar Engenharia de Features (Janela Deslizante)
all_window_features = []
if not df_raw.empty:
    sensor_axes = ['accel_x_g', 'accel_y_g', 'accel_z_g', 'gyro_x_dps', 'gyro_y_dps', 'gyro_z_dps']
    
    for state in df_raw['fan_state'].unique():
        df_state = df_raw[df_raw['fan_state'] == state].reset_index(drop=True)
        if len(df_state) < WINDOW_SIZE:
            continue
        
        print(f'Processando classe "{state}"...')
        for i in range(0, len(df_state) - WINDOW_SIZE + 1, STEP):
            window = df_state.iloc[i : i + WINDOW_SIZE]
            
            features_for_window = {'fan_state': state}
            for axis in sensor_axes:
                features_for_axis = extract_time_domain_features(window[axis])
                for feature_name, value in features_for_axis.items():
                    features_for_window[f'{axis}_{feature_name}'] = value
            
            all_window_features.append(features_for_window)

    df_features = pd.DataFrame(all_window_features)
    print(f"\n✅ Dataset de Features Criado: {df_features.shape[0]} amostras, {df_features.shape[1]-1} features.")
    display(df_features.head())
else:
    df_features = pd.DataFrame()
    print("Nenhum dado para processar.")

In [ ]:
print("--- [3/9] Seleção de Features ---")

# Com base na análise de importância do notebook 01, podemos selecionar um subconjunto de features.
# Deixe a lista VAZIA para usar TODAS as 66 features.
# Preencha com os nomes das features para treinar um modelo mais simples.

# Exemplo com as 10 features mais importantes identificadas pelo RandomForest
TOP_10_FEATURES = [
    'gyro_z_dps_std',
    'gyro_z_dps_rms',
    'gyro_y_dps_std',
    'gyro_y_dps_rms',
    'gyro_z_dps_peak',
    'gyro_y_dps_peak',
    'accel_x_g_std',
    'gyro_z_dps_root_amplitude',
    'accel_y_g_std',
    'gyro_x_dps_std'
]

# Use esta variável para controlar o treinamento.
# Para usar todas, comente a linha abaixo e descomente a seguinte.
SELECTED_FEATURES = TOP_10_FEATURES
# SELECTED_FEATURES = [] 

if SELECTED_FEATURES:
    print(f"✅ Usando um subconjunto de {len(SELECTED_FEATURES)} features selecionadas.")
else:
    print("✅ Usando todas as features disponíveis.")

In [ ]:
print("--- [4/9] Dividindo Dados em Treino e Teste ---")

if not df_features.empty:
    # Separa as features (X) do rótulo (y)
    X = df_features.drop('fan_state', axis=1)
    y = df_features['fan_state']
    
    # Aplica a seleção de features se a lista estiver preenchida
    if SELECTED_FEATURES:
        # Garante que todas as features selecionadas existem no DataFrame
        existing_selected_features = [f for f in SELECTED_FEATURES if f in X.columns]
        if len(existing_selected_features) != len(SELECTED_FEATURES):
            print("⚠️ Aviso: Algumas features selecionadas não foram encontradas no DataFrame gerado.")
        
        X = X[existing_selected_features]
        print(f"\nDataset filtrado para {X.shape[1]} features.")

    # Converte rótulos de texto para números (ex: 'LOW' -> 0), necessário para alguns modelos
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    # Divide os dados (80% para treino, 20% para teste)
    # stratify=y garante que a proporção de classes seja a mesma nos dois conjuntos
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )

    print(f"Tamanho do conjunto de treino: {len(X_train)}")
    print(f"Tamanho do conjunto de teste:  {len(X_test)}")
    print(f"Classes: {label_encoder.classes_}")
else:
    print("DataFrame de features vazio. Não é possível dividir os dados.")

In [ ]:
def plot_confusion_matrix(y_true, y_pred, class_names):
    """Plota uma matriz de confusão visualmente clara."""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Matriz de Confusão', fontsize=16)
    plt.ylabel('Classe Verdadeira', fontsize=12)
    plt.xlabel('Classe Prevista', fontsize=12)
    plt.show()

def evaluate_model(model, X_test, y_test, class_names):
    """Faz previsões, calcula métricas e plota a matriz de confusão."""
    print(f"--- Avaliando Modelo: {model.__class__.__name__} ---")
    
    # Faz previsões no conjunto de teste
    y_pred = model.predict(X_test)
    
    # Calcula a acurácia
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Acurácia no Teste: {accuracy * 100:.2f}%\n")
    
    # Mostra o relatório de classificação (precisão, recall, f1-score)
    print("Relatório de Classificação:")
    print(classification_report(y_test, y_pred, target_names=class_names))
    
    # Plota a matriz de confusão
    plot_confusion_matrix(y_test, y_pred, class_names)

print("Funções auxiliares de avaliação definidas.")

In [ ]:
print("--- [5/9] Treinando e Avaliando o Modelo de Base ---")

if 'X_train' in locals():
    # Inicializa e treina o modelo
    gnb_model = GaussianNB()
    gnb_model.fit(X_train, y_train)

    # Avalia o modelo
    evaluate_model(gnb_model, X_test, y_test, label_encoder.classes_)
else:
    print("Conjunto de treino não definido.")

In [ ]:
print("--- [6/9] Treinando e Avaliando o Modelo Random Forest ---")

if 'X_train' in locals():
    # Inicializa e treina o modelo
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)

    # Avalia o modelo
    evaluate_model(rf_model, X_test, y_test, label_encoder.classes_)
else:
    print("Conjunto de treino não definido.")

In [ ]:
print("--- [6/9] Treinando e Avaliando o Modelo LightGBM ---")

if 'X_train' in locals():
    # Inicializa e treina o modelo
    lgbm_model = lgb.LGBMClassifier(random_state=42)
    lgbm_model.fit(X_train, y_train)

    # Avalia o modelo
    evaluate_model(lgbm_model, X_test, y_test, label_encoder.classes_)
else:
    print("Conjunto de treino não definido.")

In [ ]:
print("--- [7/9] Comparando Modelos com Validação Cruzada ---")

if 'X' in locals():
    models = {
        'GaussianNB': GaussianNB(),
        'RandomForest': RandomForestClassifier(random_state=42),
        'LightGBM': lgb.LGBMClassifier(random_state=42)
    }

    results = {}
    for name, model in models.items():
        # cv=5 significa que os dados serão divididos 5 vezes
        scores = cross_val_score(model, X, y_encoded, cv=5, scoring='accuracy')
        results[name] = scores.mean()
        print(f"Acurácia Média (CV) para {name}: {results[name] * 100:.2f}%")

    # Plotar comparação
    plt.figure(figsize=(10, 6))
    sns.barplot(x=list(results.keys()), y=list(results.values()))
    plt.title('Comparação de Acurácia dos Modelos (Validação Cruzada)', fontsize=16)
    plt.ylabel('Acurácia Média', fontsize=12)
    plt.ylim(0.8, 1.0)
    plt.show()
else:
    print("Dataset completo não definido.")

In [ ]:
print("--- [8/9] Analisando a Importância das Features ---")

# Usaremos o modelo Random Forest, que já foi treinado, para esta análise
if 'rf_model' in locals():
    importances = rf_model.feature_importances_
    feature_names = X.columns
    
    # Cria um DataFrame para facilitar a ordenação e plotagem
    feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
    feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

    # Plota as 20 features mais importantes
    plt.figure(figsize=(12, 8))
    sns.barplot(x='importance', y='feature', data=feature_importance_df.head(20), palette='viridis')
    plt.title('Top 20 Features Mais Importantes (Random Forest)', fontsize=16)
    plt.xlabel('Importância', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    plt.show()
else:
    print("Modelo Random Forest não foi treinado.")

In [ ]:
print("--- [9/9] Treinando e Exportando o Modelo Final para o Dashboard ---")

def export_gaussian_nb_model(model, features, labels, output_path):
    """Exporta um modelo GaussianNB treinado no formato JSON compatível com o classifier.js"""
    
    # Calcula métricas finais
    train_acc = accuracy_score(y_encoded, model.predict(X))
    cv_scores = cross_val_score(model, X, y_encoded, cv=5)
    
    export_data = {
        "type": "gaussian_nb",
        "version": f"py_{time.strftime('%Y%m%d')}",
        "generated_at": time.strftime("%Y-%m-%d %H:%M:%S"),
        "features": features,
        "labels": list(labels),
        "priors": {},
        "stats": {},
        "metrics": {
            "train_accuracy": train_acc,
            "cv_accuracy_mean": cv_scores.mean()
        },
        "training_info": {
            "total_samples": len(X),
            "window_size": WINDOW_SIZE
        }
    }

    # Preenche Priors
    if hasattr(model, 'class_prior_'):
        priors = model.class_prior_
    else:
        priors = model.class_count_ / model.class_count_.sum()
        
    for i, label in enumerate(labels):
        export_data["priors"][label] = priors[i]
        
    # Preenche Stats (Médias e Variâncias)
    # Estrutura: stats[LABEL][FEATURE] = { mean: ..., var: ... }
    for i, label in enumerate(labels):
        export_data["stats"][label] = {}
        for j, feature in enumerate(features):
            export_data["stats"][label][feature] = {
                "mean": model.theta_[i, j],
                "var": model.var_[i, j]
            }
            
    # Salva o arquivo
    with open(output_path, 'w') as f:
        json.dump(export_data, f, indent=2)
    print(f"✅ Modelo exportado com sucesso para: {os.path.abspath(output_path)}")


if 'X' in locals():
    # NOTA: Embora Random Forest seja provavelmente melhor, o frontend atual só suporta GaussianNB.
    # Portanto, vamos treinar o GNB com todos os dados e exportá-lo.
    print("Treinando modelo GaussianNB final com todos os dados...")
    final_gnb_model = GaussianNB()
    final_gnb_model.fit(X, y_encoded)
    
    # Define o nome do arquivo de saída
    output_filename = f"gnb_model_{time.strftime('%Y%m%d')}.json"
    output_filepath = os.path.join(PATH_MODELS, output_filename)
    
    # Exporta o modelo
    export_gaussian_nb_model(final_gnb_model, list(X.columns), label_encoder.classes_, output_filepath)
    
    print("\n--- Próximos Passos ---")
    print("1. O novo modelo foi salvo na pasta 'output/models/'.")
    print("2. Copie este arquivo para a pasta 'models/' na raiz do projeto.")
    print(f"3. Atualize a constante 'MODEL_URL' no arquivo 'js/dashboard.js' para: 'models/{output_filename}'")

else:
    print("Dataset não foi criado. Não é possível treinar ou exportar o modelo.")